In [130]:
from collections import Counter
import tokenization
import preprocess
import ner
import verb_extraction
import stemming
from datetime import datetime, timedelta

In [34]:
def edit_distance (word1, word2, index1, index2):
    dp = [[0 for x in range(index2 + 1)] for x in range(index1 + 1)]
    for i in range (index1 + 1):
        for j in range (index2 + 1):
            if i == 0:
                dp[i][j] == j
            elif j == 0:
                dp[i][j] == i
            elif word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                 dp[i][j] = 1 + min ( dp[i-1][j-1], #replace
                                    dp[i-1][j],     #remove
                                    dp[i][j-1])     #insert
    return dp[index1][index2]
    

In [139]:
singleNumbers = ["واحد","اتنين ","تلاته","اربعه","خمسه","سته","سبعه","تمانيه","تسعه"]
higherNumbers = ["عشر","عشرين","تلاتين","اربعين","خمسين"]
numbersBagOfWords = []
for number in singleNumbers:
    numbersBagOfWords.append(number)    

for higher in higherNumbers:
    if higher == "عشر":
        numbersBagOfWords.append(higher + "ه")
    else:
        numbersBagOfWords.append(higher)
    for number in singleNumbers:
        if higher == "عشر" or higher == "":
            numbersBagOfWords.append(number + higher)    
        else:
            numbersBagOfWords.append(number + "و" + higher)
print(numbersBagOfWords)

['واحد', 'اتنين ', 'تلاته', 'اربعه', 'خمسه', 'سته', 'سبعه', 'تمانيه', 'تسعه', 'عشره', 'واحدعشر', 'اتنين عشر', 'تلاتهعشر', 'اربعهعشر', 'خمسهعشر', 'ستهعشر', 'سبعهعشر', 'تمانيهعشر', 'تسعهعشر', 'عشرين', 'واحدوعشرين', 'اتنين وعشرين', 'تلاتهوعشرين', 'اربعهوعشرين', 'خمسهوعشرين', 'ستهوعشرين', 'سبعهوعشرين', 'تمانيهوعشرين', 'تسعهوعشرين', 'تلاتين', 'واحدوتلاتين', 'اتنين وتلاتين', 'تلاتهوتلاتين', 'اربعهوتلاتين', 'خمسهوتلاتين', 'ستهوتلاتين', 'سبعهوتلاتين', 'تمانيهوتلاتين', 'تسعهوتلاتين', 'اربعين', 'واحدواربعين', 'اتنين واربعين', 'تلاتهواربعين', 'اربعهواربعين', 'خمسهواربعين', 'ستهواربعين', 'سبعهواربعين', 'تمانيهواربعين', 'تسعهواربعين', 'خمسين', 'واحدوخمسين', 'اتنين وخمسين', 'تلاتهوخمسين', 'اربعهوخمسين', 'خمسهوخمسين', 'ستهوخمسين', 'سبعهوخمسين', 'تمانيهوخمسين', 'تسعهوخمسين']


In [132]:
def textNumberToNumber(word):
    if word.isnumeric():
        return int(word)
    editDistance = 1000
    closestNumber = "واحد"
    for index, number in enumerate(numbersBagOfWords):
        editDistanceNew = edit_distance(word, number, len(word), len(number))
        if (editDistanceNew < editDistance or 
        (editDistanceNew == editDistance and abs( len(number) - len(word)) < abs( len(closestTextualNumber) - len(word)))):
            editDistance = editDistanceNew
            closestTextualNumber = number
            closestNumber = index + 1
    print(closestNumber)
    if editDistance >= 3:
        return 0
    return closestNumber
    
def bigrams (tokens):
    grams = []
    for index in range(len(tokens) - 1):
        grams.append([tokens[index] , tokens[index + 1]])
    return grams

In [178]:
def NLU(text):
    #Preprocessing
    text = preprocess.pre_process(text)
    #Tokenization
    tokens = tokenization.get_tokens(text)
    # NER
    ents = ner.get_ents(tokens)
    #Part of Speech and Stemming
    tokens_verb_noun = verb_extraction.extract_stem_verb(tokens,ents)
    tokens_verb_noun = stemming.stem(tokens_verb_noun)
    return text , tokens ,ents , tokens_verb_noun
text = input()
text , tokens ,ents , tokens_verb_noun = NLU(text)
for index, token in enumerate(tokens):
    if token[0] == 'ا' and token[1] == 'ل':
        tokens[index] = token[2:]
print(tokens)

['اجتماع', 'شهرين', 'مقطم', 'ساعه', '4', 'ونص', 'يوم', '25']


c:\Users\Ahmed Sherif\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\Ahmed Sherif\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LinearSVC from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\Ahmed Sherif\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [182]:
now = datetime.now()
overwriteHour, overwriteDay, overwriteMonth, morning, minuteEdit, hourEdit, dayEdit, monthEdit = 0, 0, 0, 0, 0, -2, 0, 0
bigram = bigrams(tokens)
bigram.reverse()
for gram in bigram:

    if gram[1] == "ربع" and gram[0] == "الا" and minuteEdit == 0:
        hourEdit -= 1
        minuteEdit = 45
    elif (gram[0] == "ربع" or gram[0] == "وربع") and minuteEdit == 0:
        minuteEdit = 15
    if gram[1] == "تلت" and gram[0] == "الا" and minuteEdit == 0:
        hourEdit -= 1
        minuteEdit = 40
    elif (gram[0] == "تلت" or gram[0] == "وتلت") and minuteEdit == 0:
        minuteEdit = 20
    elif (gram[0] == "نص" or gram[0] == "ونص") and minuteEdit == 0:
        minuteEdit = 30
    elif (gram[1] == "دقيقه" or gram[1] == "دقايق")and minuteEdit == 0:
        minuteEdit = textNumberToNumber(gram[0])

    if gram[0] == "ساعه" and hourEdit <= -2: 
        hourEdit += textNumberToNumber(gram[1]) + 2
        overwriteHour = 1
    elif gram[1] == "ساعه" and hourEdit <= -2:
        hourEdit += 3
    elif gram[1] == "ساعتين" and hourEdit <= -2:
        hourEdit += 4
    elif gram[1] == "ساعات" and hourEdit <= -2:
        hourEdit += textNumberToNumber(gram[0]) + 2

    if gram[0] == "يوم" and dayEdit == 0: 
        dayEdit = textNumberToNumber(gram[1]) 
        overwriteDay = 1
    elif gram[1] == "يوم" and dayEdit == 0:
        dayEdit = 1
    elif gram[1] == "يومين" and dayEdit == 0:
        dayEdit = 2
    elif gram[1] == "ايام" and dayEdit == 0:
        dayEdit = textNumberToNumber(gram[0]) 

    if gram[0] == "شهر" and monthEdit == 0: 
        monthEdit = textNumberToNumber(gram[1]) 
        overwriteMonth = 1
    elif gram[1] == "شهر" and monthEdit == 0:
        monthEdit = 1
    elif gram[1] == "شهرين" and monthEdit == 0:
        monthEdit = 2
    elif gram[1] == "شهور" and monthEdit == 0:
        monthEdit = textNumberToNumber(gram[0]) 
    
print (minuteEdit ,hourEdit, dayEdit, monthEdit, overwriteHour, overwriteDay, overwriteMonth)


# number without specifications should be an hour
# need to add "بكرة" "بعد بكرة" "الصبح" "بليل" 
# cannot deiffrentiate between "watch" and "hour" both are "ساعة"
# need to optimize the code 
# it is a static code any error will result into a distaster XD


30 4 25 2 1 1 0
